In [353]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

# Import Dataset

In [327]:
df_train = pd.read_excel('./data/Flight Prediction/Data_Train.xlsx')
df_test = pd.read_excel('./data/Flight Prediction/Test_set.xlsx')

In [328]:
df_train.shape, df_test.shape

((10683, 11), (2671, 10))

In [329]:
# Combine the dataset
final_df = pd.concat([df_train, df_test])

In [330]:
final_df.shape

(13354, 11)

In [331]:
final_df.tail()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
2666,Air India,6/06/2019,Kolkata,Banglore,CCU → DEL → BLR,20:30,20:25 07 Jun,23h 55m,1 stop,No info,NaN
2667,IndiGo,27/03/2019,Kolkata,Banglore,CCU → BLR,14:20,16:55,2h 35m,non-stop,No info,NaN
2668,Jet Airways,6/03/2019,Delhi,Cochin,DEL → BOM → COK,21:50,04:25 07 Mar,6h 35m,1 stop,No info,NaN
2669,Air India,6/03/2019,Delhi,Cochin,DEL → BOM → COK,04:00,19:15,15h 15m,1 stop,No info,NaN
2670,Multiple carriers,15/06/2019,Delhi,Cochin,DEL → BOM → COK,04:55,19:15,14h 20m,1 stop,No info,NaN


In [332]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


In [333]:
final_df.isna().sum()

Airline               0
Date_of_Journey       0
Source                0
Destination           0
Route                 1
Dep_Time              0
Arrival_Time          0
Duration              0
Total_Stops           1
Additional_Info       0
Price              2671
dtype: int64

# Feature Engineering

In [334]:
# Extract Date, Month, Year from `Date_of_journey`

final_df['Date_of_Journey'] = pd.to_datetime(final_df['Date_of_Journey'])
final_df['Date'] = final_df['Date_of_Journey'].dt.day
final_df['Month'] = final_df['Date_of_Journey'].dt.month
final_df['Year'] = final_df['Date_of_Journey'].dt.year
final_df.drop(['Date_of_Journey'], axis=1, inplace=True)

/tmp/ipykernel_73390/828559845.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  final_df['Date_of_Journey'] = pd.to_datetime(final_df['Date_of_Journey'])


## Extract Hour and Minute from Arrival_Time

In [335]:
final_df['Arrival_Time'] = final_df['Arrival_Time'].str.split(' ').str[0]

final_df['Arrival_Hour'] = final_df['Arrival_Time'].apply(lambda x:x.split(':')[0]).astype(int)
final_df['Arrival_Minute'] = final_df['Arrival_Time'].apply(lambda x:x.split(':')[1]).astype(int)
final_df.drop(['Arrival_Time'], axis=1, inplace=True)

## Extract Hour and Minute from Dep_Time

In [336]:
final_df['Dep_Hour'] = final_df['Dep_Time'].apply(lambda x:x.split(':')[0]).astype(int)
final_df['Dep_Minute'] = final_df['Dep_Time'].apply(lambda x:x.split(':')[1]).astype(int)
final_df.drop(['Dep_Time'], axis=1, inplace=True)

In [337]:
## Explore how much i need to fill null values based on Duration
final_df[final_df['Total_Stops'].isna()]

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute
9039,Air India,Delhi,Cochin,NaN,23h 40m,NaN,No info,7480.0,5,6,2019,9,25,9,45


In [338]:
filtered_df = final_df[(final_df['Source'] == 'Delhi') & 
                       (final_df['Destination'] == 'Cochin') & 
                       (final_df['Airline'] == 'Air India')]

hours_extracted = filtered_df['Duration'].str.extract(r'(\d+)h')[0].astype(int)

condition = (hours_extracted > 20) & ~(filtered_df['Price'].isna()) & (filtered_df['Total_Stops'] == '1 stop')
filtered_df[condition][['Airline', 'Source', 'Destination', 'Duration', 'Total_Stops', 'Price']]

,Airline,Source,Destination,Duration,Total_Stops,Price
61,Air India,Delhi,Cochin,27h 20m,1 stop,5117.0
191,Air India,Delhi,Cochin,25h 35m,1 stop,7165.0
254,Air India,Delhi,Cochin,24h 15m,1 stop,6587.0
274,Air India,Delhi,Cochin,27h 20m,1 stop,4487.0
281,Air India,Delhi,Cochin,27h 20m,1 stop,14042.0
...,...,...,...,...,...,...
10377,Air India,Delhi,Cochin,26h 30m,1 stop,8119.0
10478,Air India,Delhi,Cochin,23h 40m,1 stop,7532.0
10559,Air India,Delhi,Cochin,22h 15m,1 stop,8372.0
10566,Air India,Delhi,Cochin,27h 20m,1 stop,4487.0


## Route annd Total_Stops are related to each other

In [339]:
## So we drop Route
### Clean Total_Stops data

final_df.drop(["Route"], axis=1, inplace=True)
final_df["Total_Stops"] = final_df["Total_Stops"].map(
    {"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4, "nan": 1}
)
final_df["Total_Stops"] = final_df["Total_Stops"].fillna(1)

In [340]:
final_df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute
0,IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662.0,5,1,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,19h,2.0,No info,13882.0,6,9,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218.0,5,12,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302.0,3,1,2019,21,35,16,50


## Convert Duration into minutes

In [341]:
# Convert Duration into minutes
final_df[['duration_hour', 'duration_minutes']] = final_df['Duration'].str.split(' ', expand=True)
final_df['duration_minutes'].fillna(final_df['duration_hour'], inplace=True)

final_df['duration_hour'] = final_df['duration_hour'].str.strip('h') 
final_df['duration_hour'] = pd.to_numeric(final_df['duration_hour'], errors='coerce')

final_df['duration_minutes'] = final_df['duration_minutes'].str.strip('m') 
final_df['duration_minutes'] = pd.to_numeric(final_df['duration_minutes'], errors='coerce')

final_df['duration_minutes'] = final_df['duration_hour']*60+final_df['duration_minutes']

final_df.drop(['Duration' ,'duration_hour'], axis=1, inplace=True)

## OneHotEncoding

In [342]:
from sklearn.preprocessing import OneHotEncoder
import joblib
# final_df = pd.get_dummies(final_df, columns=['Airline', 'Source', 'Destination'])

In [343]:
onehot_encoder = OneHotEncoder()

columns_to_encode = ['Airline', 'Source', 'Destination', 'Additional_Info']
df_to_encode = final_df[columns_to_encode]

In [344]:
encoded_data = onehot_encoder.fit_transform(df_to_encode)

# Save Encoder
joblib.dump(onehot_encoder, 'models/onehot_encoder.pkl')
loaded_onehot_encoder = joblib.load('models/onehot_encoder.pkl')

# Make into df
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=loaded_onehot_encoder.get_feature_names_out(columns_to_encode))

# Reset index
encoded_df.reset_index(drop=True, inplace=True)
final_df.reset_index(drop=True, inplace=True)

# Concatenate
final_df = pd.concat([final_df, encoded_df], axis=1)

# Drop old columns
final_df.drop(columns_to_encode, axis=1, inplace=True)

In [376]:
final_df.head()

,Total_Stops,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,duration_minutes,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy,Source_Banglore,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi,Additional_Info_1 Long layover,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,0.0,3897.0,24,3,2019,1,10,22,20,170.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2.0,7662.0,5,1,2019,13,15,5,50,445.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.0,13882.0,6,9,2019,4,25,9,25,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,6218.0,5,12,2019,23,30,18,5,325.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,13302.0,3,1,2019,21,35,16,50,285.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Testing One Hot Encoder

In [374]:
data = {
    'Airline': ['IndiGo'],
    'Source': ['Banglore'],
    'Destination': ['New Delhi'],
    'Additional_Info': ['No info']
}

test = pd.DataFrame(data)

In [375]:
encoded_data = loaded_onehot_encoder.transform(test)

In [370]:
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=loaded_onehot_encoder.get_feature_names_out(columns_to_encode))

In [371]:
encoded_df

,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy,Source_Banglore,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi,Additional_Info_1 Long layover,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
